In [23]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from bayes_opt import BayesianOptimization
import catboost as cb
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

# 解决中文乱码问题
plt.rcParams["font.sans-serif"]=["SimHei"] 
plt.rcParams["axes.unicode_minus"]=False

In [2]:
data=pd.read_excel('./use.xlsx')
train_data,test_data=train_test_split(data,test_size=0.2, random_state=1314)
feature=train_data.drop(['Y','Unnamed: 0'],axis=1)
label=train_data['Y']

In [3]:
feature

,Cr,Zr,Pt,In,Zn,K,Co,Mg,Ni,V,...,SBA-1,Al2O3,温度(_摄氏度),时间(h),循环,流量（丙烷）,流量（氢气）,流量（CO2）,流量（保护气）,WHSV(流量)
10,0.00,0.0,0.40,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.00,0.00,600,17.00,2,24.00,0.0,0.0,76.0,1.70
284,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,12.0,...,0.00,88.00,600,0.33,1,28.00,28.0,0.0,44.0,3.30
464,0.00,0.0,0.00,0.0,7.0,0.00,0.0,0.0,0.0,0.0,...,0.00,0.00,600,0.10,1,5.00,0.0,0.0,95.0,0.59
0,0.00,0.0,0.42,0.0,0.0,0.65,0.0,0.0,0.0,0.0,...,0.00,0.00,600,65.00,1,24.00,0.0,0.0,76.0,1.70
571,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.00,97.00,620,0.25,7,100.00,0.0,0.0,0.0,6.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,0.00,0.0,0.10,0.0,0.0,0.25,0.0,0.0,0.0,0.0,...,0.00,96.65,620,0.25,1,100.00,0.0,0.0,0.0,6.47
103,0.00,0.0,0.48,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.00,0.00,550,1.00,1,29.00,14.0,0.0,57.0,14.00
168,0.04,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,99.96,0.00,500,0.16,1,6.67,0.0,33.3,60.0,6.47
251,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,4.9,...,0.00,0.00,550,0.10,1,40.00,0.0,0.0,60.0,1.78


In [28]:
# 预测用数据1
l1=list([33,34,40])
data_predict = train_data.copy()
data_predict=data_predict.drop(['Unnamed: 0'],axis=1)
data_predict.iloc[:, :] = 0
data_6 = pd.DataFrame(np.zeros((10000, 42)))
data_6.columns = data_predict.columns
data_predict = pd.concat([data_predict, data_6])
for i in l1:
    data_predict.iloc[:, i] = feature.iloc[:, i].mean()
data_predict.iloc[:, 35] = 1
data_predict.iloc[:, 36] = 10
data_predict.iloc[:, 37] = 20
data_predict.iloc[:, 38] = 10
data_predict.iloc[:, 39] = 60
for i in range(10471):
    x_1 = random.randint(0, 20)
    data_predict.iloc[i, random.randint(0, 12)] = x_1
    data_predict.iloc[i, random.randint(13, 32)] = 100 - x_1

In [29]:
data_predict

,Cr,Zr,Pt,In,Zn,K,Co,Mg,Ni,V,...,Al2O3,温度(_摄氏度),时间(h),循环,流量（丙烷）,流量（氢气）,流量（CO2）,流量（保护气）,WHSV(流量),Y
10,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,587.165605,8.213631,1,10,20,10,60,9.168429,0.0
284,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,...,0.0,587.165605,8.213631,1,10,20,10,60,9.168429,0.0
464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,587.165605,8.213631,1,10,20,10,60,9.168429,0.0
0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,587.165605,8.213631,1,10,20,10,60,9.168429,0.0
571,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,...,0.0,587.165605,8.213631,1,10,20,10,60,9.168429,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,587.165605,8.213631,1,10,20,10,60,9.168429,0.0
9996,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,587.165605,8.213631,1,10,20,10,60,9.168429,0.0
9997,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,...,0.0,587.165605,8.213631,1,10,20,10,60,9.168429,0.0
9998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,...,0.0,587.165605,8.213631,1,10,20,10,60,9.168429,0.0


In [ ]:
# 预测用数据1
l1=list([50,51,57])
data_predict1 = train_data.copy()
data_predict1=data_predict1.drop(['Unnamed: 0'],axis=1)
data_predict1.iloc[:, :] = 0
data_5 = pd.DataFrame(np.zeros((10000, 59)))
data_5.columns = data_predict1.columns
data_predict1 = pd.concat([data_predict1, data_5])
for i in l1:
    data_predict1.iloc[:, i] = feature.iloc[:, i].mean()
data_predict1.iloc[:, 52] = 2
data_predict1.iloc[:, 53] = 20
data_predict1.iloc[:, 54] = 20
data_predict1.iloc[:, 55] = 10
data_predict1.iloc[:, 56] = 50
for i in range(10491):
    x_1 = random.randint(0, 20)
    x_2 = random.randint(0, 20)
    a1 = random.randint(0, 15)
    data_predict1.iloc[i, a1] = x_1
    data_predict1.iloc[i, random.randint(0, a1)] = x_2
    data_predict1.iloc[i, random.randint(16, 49)] = 100 - x_1 - x_2

In [ ]:
data_predict1

,Zr,Ni,Zn,Mg,Na,Pt,Cr,Mn,K,Co,...,Z-5,温度(_摄氏度),时间(h),循环,流量（丙烷）,流量（氢气）,流量（CO2）,流量（保护气）,WHSV(流量),Y
344,17.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,587.749491,8.744053,1,10,10,10,70,9.136456,0.0
530,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,587.749491,8.744053,1,10,10,10,70,9.136456,0.0
121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,587.749491,8.744053,1,10,10,10,70,9.136456,0.0
252,0.0,0.0,16.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,587.749491,8.744053,1,10,10,10,70,9.136456,0.0
4,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,587.749491,8.744053,1,10,10,10,70,9.136456,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,12.0,...,0.0,587.749491,8.744053,1,10,10,10,70,9.136456,0.0
9996,0.0,9.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,...,0.0,587.749491,8.744053,1,10,10,10,70,9.136456,0.0
9997,0.0,0.0,11.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,587.749491,8.744053,1,10,10,10,70,9.136456,0.0
9998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,587.749491,8.744053,1,10,10,10,70,9.136456,0.0


In [30]:
# 结果预测（一元催化剂）
model = xgb.XGBRegressor(n_estimators=10000,random_state=1314,booster='gbtree',max_delta_step=100,reg_lambda=2.001,learning_rate=0.07).fit(X=feature , y=label)
y_hat = pd.DataFrame(model.predict(np.array(data_predict.drop(['Y'],axis=1))))
data_predict['y_hat'] = y_hat
data_predict.sort_values(by='y_hat', inplace=True, ascending=False)
data_predict.to_excel('一元催化剂.xlsx')

In [16]:
# 结果预测（二元催化剂）
model = cb.CatBoostRegressor(iterations=970,subsample=0.58,max_depth=10,l2_leaf_reg=3.680600849975441,silent=True,random_state=1314,learning_rate=0.25850134121802654).fit(X=feature , y=label)
y_hat = pd.DataFrame(model.predict(np.array(data_predict1.drop(['Y'],axis=1))))
data_predict1['y_hat'] = y_hat
data_predict1.sort_values(by='y_hat', inplace=True, ascending=False)
data_predict1.to_excel('二元催化剂.xlsx')

NameError: name 'cb' is not defined